In [2]:
import os
import sys

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.initializers import Constant

import numpy as np
import csv
import pandas as pd

## Prepare data

In [ ]:
with open ("../data/ml-100k/u.data", "r") as f:
    data = list(csv.reader(f, delimiter="\t"))
data = np.array(data)
film_dim = np.amax(np.array(data[:,1]).astype(np.float))
user_dim = np.amax(np.array(data[:,0]).astype(np.float))
print("film size", film_dim)
print("user size", user_dim)
data = data.astype(np.int)
data = pd.DataFrame(data)
data.sort_values([0,3],inplace=True) ## Sort data
data.rename(columns= {0: 'user_id',
                      1: 'movie_id',
                      2: 'rating',
                      3: 'time'},
            inplace=True)
data_x = []
data_y = []
max_len = 0
for i in range(1, user_dim.astype(np.int)+1):
    user = data[data.user_id == i]
    if len(user) > max_len:
        max_len = len(user)
    data_x.append(user['movie_id'])
    data_y.append(user['rating'])

data_x =  tf.keras.preprocessing.sequence.pad_sequences(data_x,padding='post',maxlen=max_len)
data_y =  tf.keras.preprocessing.sequence.pad_sequences(data_y,padding='post',maxlen=max_len)

train_x = data_x[:700]
test_x = data_x[700:]
train_y = data_y[:700]
test_y = data_y[700:]

### Embeddings size
While LSTM units = 64 and movies were not order by time of rating

| Hyperparamater-value | Accuracy | Test Accuracy |
| -------------------- | -------- | ------------- |
| 64                   | 46.94%   | 35.22%        |
| 128                  | 50.09%   | 34.63%        |
| 256                  | 53.71%   | 34.13%        |
| 512                  | 58.82%   | 34.34%        |

### LSTM Units
While embedding size was 300 and movies were not order by time of rating

| Hyperparamater-value | Accuracy | Test Accuracy |
| -------------------- | -------- | ------------- |
| lstm_units (32)      | 51.75%   | 33.73%        |
| lstm_units (64)      | 53.68%   | 33.22%        |
| lstm_units (128)     | 53.52%   | 33.76%        |

### Sort data by time

LSTM unit = 64
Embeddings size = 512
Accuracy: 63.40%
Test Accuracy: 32.43%

In [ ]:
class POSTagger(keras.Model):

    def __init__(self):
        super(POSTagger, self).__init__()
        # FIXME
        lstm_units = 64
        num_classes = 6
        self.emb = Embedding(
            input_dim=1683 + 1,
            output_dim=512,
            mask_zero=True,
            trainable=True)

        self.lstm = Bidirectional(LSTM(
            units=lstm_units, return_sequences=True))

        self.dense = Dense(
            units=num_classes,
            activation='softmax')

    def call(self, x):
        # FIXME
        mask = self.emb.compute_mask(x)
        x = self.emb(x)
        x = self.lstm(x, mask=mask)
        x = self.dense(x)
        return x


model = POSTagger()

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(
    x=train_x,
    y=train_y,
    batch_size=10,
    epochs=30,
    validation_data=(test_x, test_y))